In [25]:
import numpy as np
import pandas as pd
np.random.seed(8)
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from keras.models import Model
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback

import warnings
warnings.filterwarnings('ignore')

train = pd.read_csv('train.csv')
test_cm = pd.read_csv('test.csv')
test_lb = pd.read_csv('test_labels.csv')
test_all = pd.merge(test_cm, test_lb, on='id')
test = test_all[test_all['toxic'] != -1]

In [26]:
X_train = train["comment_text"].values
y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_test = test["comment_text"].values
y_test = test[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values

In [27]:
vocab = 100000
maxlen = 200
embed_size = 300

In [28]:
t = text.Tokenizer(num_words=vocab)
t.fit_on_texts(list(X_train) + list(X_test))

In [29]:
X_train = t.texts_to_sequences(X_train)
X_test = t.texts_to_sequences(X_test)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [30]:
EMBEDDING_FILE = 'crawl-300d-2M.vec'
embeddings_index = dict()
f = open(EMBEDDING_FILE)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s pre-trained words' % len(embeddings_index))


Loaded 2000000 pre-trained words


In [31]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab, embed_size))
for word, i in t.word_index.items():
    if i >= vocab: 
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [32]:
num_filters = 32
activation = 'tanh'
inp = Input(shape=(maxlen, ))
x = Embedding(vocab, embed_size, weights=[embedding_matrix], input_length=maxlen, trainable=False)(inp)
x = SpatialDropout1D(0.4)(x)
x = Reshape((maxlen, embed_size, 1))(x)

conv_0 = Conv2D(num_filters, kernel_size=(2, embed_size), activation=activation)(x)
maxpool_0 = MaxPool2D(pool_size=(maxlen - 1, 1))(conv_0)
conv_1 = Conv2D(num_filters, kernel_size=(3, embed_size), activation=activation)(x)
maxpool_1 = MaxPool2D(pool_size=(maxlen - 2, 1))(conv_1)
conv_2 = Conv2D(num_filters, kernel_size=(4, embed_size), activation=activation)(x)
maxpool_2 = MaxPool2D(pool_size=(maxlen - 3, 1))(conv_2)
conv_3 = Conv2D(num_filters, kernel_size=(5, embed_size), activation=activation)(x)
maxpool_3 = MaxPool2D(pool_size=(maxlen - 4, 1))(conv_3)
#conv_4 = Conv2D(num_filters, kernel_size=(6, embed_size), activation=activation)(x)
#maxpool_4 = MaxPool2D(pool_size=(maxlen - 5, 1))(conv_4)

y = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2, maxpool_3])
y = Flatten()(y)
y = Dropout(0.1)(y)

outp = Dense(6, activation="sigmoid")(y)

model = Model(inputs=inp, outputs=outp)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 200, 300)     30000000    input_2[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_2 (SpatialDro (None, 200, 300)     0           embedding_2[0][0]                
__________________________________________________________________________________________________
reshape_2 (Reshape)             (None, 200, 300, 1)  0           spatial_dropout1d_2[0][0]        
__________________________________________________________________________________________________
conv2d_5 (

In [33]:
x_trainS, x_val, y_trainS, y_val = train_test_split(x_train, y_train, train_size=0.90, random_state=8)

In [36]:
class RocAuc(Callback):

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.validation_data[0], verbose=0)
        score = roc_auc_score(self.validation_data[1], y_pred)
        print("\n roc_auc - epoch: %d - score: %.6f \n" % (epoch+1, score))

In [37]:
RocAuc = RocAucEvaluation(validation_data=(x_val, y_val), interval=1)

In [38]:
from keras.callbacks import ModelCheckpoint
batch_size=256
epochs = 5
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.pre_trained.conv2.testscore.hdf5', verbose=1, save_best_only=True)
#RocAuc = RocAuc()

In [39]:
h = model.fit(x_trainS, y_trainS, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val),
                 callbacks=[checkpointer, RocAuc], verbose=2)

Train on 143613 samples, validate on 15958 samples
Epoch 1/5
 - 500s - loss: 0.0817 - acc: 0.9749 - val_loss: 0.0474 - val_acc: 0.9825

Epoch 00001: val_loss improved from inf to 0.04743, saving model to saved_models/weights.best.pre_trained.conv2.testscore.hdf5

 ROC-AUC - epoch: 1 - score: 0.979381 

Epoch 2/5
 - 508s - loss: 0.0505 - acc: 0.9812 - val_loss: 0.0433 - val_acc: 0.9832

Epoch 00002: val_loss improved from 0.04743 to 0.04333, saving model to saved_models/weights.best.pre_trained.conv2.testscore.hdf5

 ROC-AUC - epoch: 2 - score: 0.986268 

Epoch 3/5
 - 506s - loss: 0.0467 - acc: 0.9822 - val_loss: 0.0418 - val_acc: 0.9837

Epoch 00003: val_loss improved from 0.04333 to 0.04182, saving model to saved_models/weights.best.pre_trained.conv2.testscore.hdf5

 ROC-AUC - epoch: 3 - score: 0.987694 

Epoch 4/5
 - 506s - loss: 0.0449 - acc: 0.9827 - val_loss: 0.0413 - val_acc: 0.9841

Epoch 00004: val_loss improved from 0.04182 to 0.04126, saving model to saved_models/weights.best

In [40]:
y_pred = model.predict(x_test, batch_size=256)

In [41]:
score = roc_auc_score(y_test, y_pred)
print("\n roc_auc score: %.6f \n" % (score))


 roc_auc score: 0.981911 

